# Lab06 - Intuitionistic propositional logic in Agda

**LICENCE:** This tutorial contains adaptations of material from [Programming Language Foundations in Agda](https://plfa.github.io/) by Phil Wadler and Wen Kokke. It is licensed under Creative Commons Attribution 4.0 International.

**SYNTAX:** You can enter `→` by writing `\->` and pressing TAB. Similar useful combinations include `\lambda`, `\neg`, `\top`, `\bot`, `\and`, `\or`, `\forall`, `\exists`, `\Pi`, \\`Sigma`, ...

In this lab we will explore intuitionistic propositional logic via the Agda programming language. Agda can be seen as Haskell on steroids with a more powerful type system.

# Implication

Intuitionistic implication is implemented as *function space* $A \to B$. The idea is that a proof of $A \to B$ is a (terminating) *program* $t : A \to B$ that, given a proof $a : A$ of $A$, always terminates and produces a proof $(t\; a) : B$ of $B$.

For instance, the following Agda program `id` proves the intuitionistic tautology $A \to A$:

In [ ]:
module AgdaLab01 where

id : {A : Set} -> A -> A
id a = a

-- the same with λ-abstraction:
id' id'' : {A : Set} -> A -> A
id' = λ a → a
id'' = \ a -> a -- alternative non-unicode syntax

Compare this with the equivalent Haskell definition:

```haskell no-exec id=8143ef57-8378-4252-8cf8-8a368c1de1f2
id :: A → A
id x = x
```

In Agda we need to explicitly say what `A` is with the *implicit argument* `{A : Set}`. (In Agda, `Set` means *type*.) Thus `id` is a higher order function that takes as (implicit) input a type `A`, a value of type `A`, and returns a value also of type `A`. This shows how already such a simple function uses a dependent type.

The $\to$-elimination rule from natural deduction is *function application* and the $\to$-introduction rule is $\lambda$-abstraction:

In [ ]:
→-elim : {A B : Set} → (A → B) → A → B
→-elim a→b a = a→b a

**SPACING:** Agda allows us to be very flexible in the variable names, which can be strings such as `x`, `idλ`, `a→b`, `&&true`, or even arbitrary unicode symbols `💔`, *provided there is no space in-between*. As a consequence, *spaces in Agda have a syntactic meaning as separators*, and we need to be very generous with them.

## **Exercise**

Prove in Agda following theorems of intuitionistic propositional logic:

$\begin{align}
	(1) &&& A \to B \to A, \\
	(2) &&& (A \to B \to C) \to (A \to B) \to A \to C, \\
  (3) &&& (A \to B) \to (A \to C) \to (B \to C \to D) \to A \to D, \\
  (4) &&& A \to A \to A. \end{align}$

 How many proofs does $(4)$ have?

In [ ]:
A1 : {A B : Set} → A → B → A
A1 a b = a

A2 : {A B C : Set} → (A → B → C) → (A → B) → A → C
A2 a→b→c a→b a = a→b→c a (a→b a)

diamond : {A B C D : Set} → (A → B) → (A → C) → (B → C → D) → A → D
diamond a→b a→c b→c→d a = b→c→d (a→b a) (a→c a)

proj : {A : Set} → A → A → A
proj a₁ a₂ = a₁ -- or we can return a₂

# True and false

The two truth values $\top$ and $\bot$ are implemented via Agda's data type mechanism. The intuition is that the type $\top$ has precisely one inhabitant (called `tt` below) and that $\bot$ has no inhabitants at all.

In [ ]:
data ⊤ : Set where
  tt : ⊤
  
data ⊥ : Set where

The ⊥-elimination rule is implemented via the absurd pattern "()":

In [ ]:
⊥-elim : {A : Set} → ⊥ → A
⊥-elim ()

f : {A B : Set} → B → ⊥ → A
f _ absurd = ⊥-elim absurd

# Negation

Negation is not a primitive in intuitionistic logic. In intuitionistic logic $\neg A$ means that, if we had a proof of $A$, then we could derive a contradiction $\bot$:

$\neg A \;\equiv\; A \to \bot.$

In [ ]:
infix 3 ¬_

¬_ : Set → Set
¬ A = A → ⊥

Note how `¬_` is a type-level function, i.e., a function mapping types to types!

## Exercise

Use `⊥-elim` to prove $\neg A \to A \to B$:

In [ ]:
exfalso : {A B : Set} → ¬ A → A → B
exfalso notA a = ⊥-elim (notA a)

## Exercise

The logic of Agda is intuitionistic. In particular, in Agda the following double negation law does *not* hold:

$A \leftrightarrow \neg \neg A.$

Which one of the two directions holds in intuitionistic logic?

* Formalise this and prove it in Agda. Recall that ¬ ¬ A = (A → ⊥) → ⊥.
* Does the proof (i.e., program) resemble something we have already seen?

In [ ]:
-- ¬¬-elim does not hold
¬¬-elim : {A : Set} → ¬ ¬ A → A
¬¬-elim f = ?

¬¬-intro : {A : Set} → A → ¬ ¬ A
¬¬-intro a f = ⊥-elim (f a)

## **Exercise**

Show that the following *triple negation* law holds intuitionistically. 

In [ ]:
triple-negation : {A : Set} → ¬ ¬ ¬ A → ¬ A
triple-negation notNotNotA a = notNotNotA (λ notA → ⊥-elim (notA a))

## **Exercise**

The *contrapositive* of an implication $A \to B$ is $\neg B \to \neg A$. In classical logic an implication and its contrapositive are logically equivalent, i.e., the following is a tautology:

$(A \to B) \leftrightarrow (\neg B \to \neg A).$

Prove which, if any, of the two directions above holds in intuitionistic logic.

In [ ]:
contrapositive : {A B : Set} → (A → B) → (¬ B → ¬ A)
contrapositive a→b notB a = notB (a→b a)

# Conjunction

According to the BHK interpretation, a proof of $A \wedge B$ is a pair $\langle a, b \rangle$, where $a$ is a proof of $A$ and $b$ is a proof of $B$. This intuition translates immediately into the following *product* datatype:

In [ ]:
infix 2 _∧_ 

-- _∧_ is the cartesian product operator ×

data _∧_ (A : Set) (B : Set) : Set where
  _,_ : A → B → A ∧ B

The datatype `_∧_` is parametrised by two types, `A` and `B`, and it is written `A ∧ B` or, using prefix notation, `_∧_ A B`. It has only one constructor `_,_`, corresponding to the ∧-introduction rule: Given elements `a : A` and `b : B`,  `a , b` has type `A ∧ B` (notice the mandatory spaces!). The term above can also be written in prefix notation as `_,_ A B`.

We can then define the two projection functions, which correspond to the two ∧-elimination rules:

In [ ]:
fst : {A B : Set} → A ∧ B → A
fst (a , _) = a

snd : {A B : Set} → A ∧ B → B
snd (_ , b) = b

## **Exercise**

Formalise and prove the following:

1. Conjunction is commutative.


1. Curry/uncurry: The formula/type $A → B → C$ is "the same" as $A \wedge B → C$.

In [ ]:
∧-commutative : {A B : Set} → A ∧ B → B ∧ A
∧-commutative (a , b) = (b , a)

uncurry : {A B C : Set} → (A → B → C) → A ∧ B → C
uncurry a→b→c (a , b) = a→b→c a b

curry : {A B C : Set} → (A ∧ B → C) → A → B → C
curry a∧b→c a b = a∧b→c (a , b)

# Disjunction

According to the BHK interpretation, a proof of $A_1 \vee A_2$ is a pair $(k, t_k)$, where $k \in \{1, 2\}$ specifies that we are proving $A_k$ and $t_k : A_k$ is a proof thereof. Since the first component is finite, disjunction can be implemented with two constructors. Each constructor corresponds to a ∨-introduction rule.

In [ ]:
-- we give disjunction lower priority than conjunction
-- so we can omit parenthesis from (A ∧ B) ∨ C and write A ∧ B ∨ C instead.
infix 1 _∨_ 

data _∨_ (A : Set) (B : Set) : Set where
    left : A → A ∨ B
    right : B → A ∨ B

We can do case analysis by pattern matching on the constructor. This corresponds to the ∨-elimination rule:

In [ ]:
case : {A B C : Set} → (A → C) → (B → C) → A ∨ B → C
case f g (left a) = f a
case f g (right b) = g b

## **Exercise**

In classical logic we have the following *law of excluded middle*: 

$A \vee \neg A.$

1. Why there is no Agda program of the corresponding type `{A : Set} → A ∨ ¬ A`?
2. *Challenge*: Write an Agda program for the *irrefutability* of the law of excluded middle: $\neg \neg (A \vee \neg A).$

   Hint: Expand the definition of $\neg$. You will need: `left`, `right`, and $\lambda$-abstraction.

In [ ]:
-- ¬ ¬ (A ∨ ¬ A) = ((A ∨ (A → ⊥)) → ⊥) → ⊥ 

irrefutability : {A : Set} → ¬ ¬ (A ∨ ¬ A)
irrefutability notA∨¬A = notA∨¬A (right (λ a → notA∨¬A (left a)))

## **Exercise (De Morgan Laws)**

Are the following laws valid in intuitionistic logic? If so, write a proof in Agda.

$\begin{align}
(1) \qquad \neg (A \vee B) \leftrightarrow \neg A \wedge \neg B. \\
(2) \qquad \neg A \vee \neg B \to \neg (A \wedge B). \\
(3) \qquad \neg (A \wedge B) \to \neg A \vee \neg B. \end{align}$

In [ ]:
de_morgan1-1 : {A B : Set} → ¬ (A ∨ B) → ¬ A ∧ ¬ B
de_morgan1-1 notA∨B = ((λ a → notA∨B (left a)), (λ b → notA∨B (right b)))

de_morgan1-2 : {A B : Set} → ¬ A ∧ ¬ B → ¬ (A ∨ B)
de_morgan1-2 (notA , notB) a∨b = case notA notB a∨b

de_morgan2 : {A B : Set} → ¬ A ∨ ¬ B → ¬ (A ∧ B)
de_morgan2 ¬a∨¬b (a , b) = case (λ notA → notA a) (λ notB → notB b) ¬a∨¬b

-- de_morgan3 does not hold
de_morgan3 : {A B : Set} → ¬ (A ∧ B) → ¬ A ∨ ¬ B
de_morgan3 = ?

## **Exercise**

In classical logic, $A \to B$ is logically equivalent to $\neg A \vee B$. Which of the following two directions hold in intuitionistic logic? Prove it in Agda.

$(A \to B) \leftrightarrow (\neg A \vee B).$

In [ ]:
rightToLeft : {A B : Set} → (¬ A ∨ B) → A → B
rightToLeft ¬a∨b a = case (λ notA → ⊥-elim (notA a)) id ¬a∨b

# Challenges

The following exercises are entirely optional and not mandatory to be completed. Nonetheless, they show that very interesting phenomena happen in intuitionistic logic already at the propositional level.

## Weak Peirce's law

We know that Peirce's law is not an intuitionistic tautology. Show that the following related formula is an intuitionistic tautology:

$((((𝐴→𝐵)→𝐴)→𝐴)→𝐵)→𝐵.$

In [ ]:
wp : {A B : Set} → ((((A → B) → A) → A) → B) → B
wp f = f (λ g → g (λ h → f (λ _ → h)))

## Intuitionistically equivalent LEM formulations

In the previous exercises we have seen that the following principles are not intuitionistic tautologies:

1. Law of excluded middle: A∨¬A.
2. Elimination of double negation: ¬¬A→A.
3. Implication as disjunction: (A→B)→¬A∨B.
4. The Negated De Morgan's law: ¬(¬A∧¬B)→A∨B.
5. Peirce's Law: ((A→B)→A)→A.

Show that all principles above are logically equivalent in intuitionistic logic. Each propositional variable A,B is universally quantified in each principle.

*Hint:* Prove the following sequence of implications:

* 1→2.
* 2→3: Use irrefutability of (A→B)→¬A∨B, proved earlier: ¬¬((A→B)→¬A∨B).
* 3→1.
* 1→4: Use the excluded middle for 𝐴 and for 𝐵.
* 4→1: Use ¬(¬A∧¬B)→A∨B with B≡¬A.
* 1→5.
* 5→1: Use Peirce's law ((A′→B′)→A′)→A′ with A′≡A∨¬A and 𝐵′≡⊥.

In [ ]:
1→2 : {A : Set} → A ∨ ¬ A → (¬ ¬ A → A)
1→2 a∨¬a notNotA = case id (λ notA → ⊥-elim (notNotA notA)) a∨¬a

2→3 : {A B : Set} → (¬ ¬ (¬ A ∨ B) → ¬ A ∨ B) → ((A → B) → ¬ A ∨ B)
2→3 f a→b = f (λ g → g (left (λ a → g (right (a→b a)))))

3→1 : {A : Set} → ((A → A) → ¬ A ∨ A) → A ∨ ¬ A
3→1 f = case (λ notA → right notA) (λ b → left b) (f id)

1→4 : {A B : Set} → A ∨ ¬ A → B ∨ ¬ B → (¬ (¬ A ∧ ¬ B) → A ∨ B)
1→4 a∨¬a b∨¬b f = case
  (λ a → left a)
  (λ notA → right (case id (λ notB → ⊥-elim (f (notA , notB))) b∨¬b))
  a∨¬a

4→1 : {A : Set} → (¬ (¬ A ∧ ¬ ¬ A) → A ∨ ¬ A) → A ∨ ¬ A
4→1 f = f (λ g → (snd g) (fst g))

1→5 : {A B : Set} → A ∨ ¬ A → (((A → B) → A) → A)
1→5 a∨¬a f = case id (λ notA → f (λ a → ⊥-elim (notA a))) a∨¬a

5→1 : {A : Set} → (((A ∨ ¬ A → ⊥) → A ∨ ¬ A) → A ∨ ¬ A) → A ∨ ¬ A
5→1 f = f (λ g → right (λ a → g (left a)))